## DeepSpeed API

这一节介绍DeepSpeed所提供的API，主要关注ZeRO零冗余优化器

零冗余优化器 (ZeRO) 通过在分布式代码运行中将三个模型状态（优化器状态、梯度和参数）划分到不同GPU上来消除并行进程的内存冗余。通过这样做，与单纯数据并行相比提高了内存效率，同时保留了其计算粒度和通信效率。

1. **ZeRO Stage 1**：优化器状态（例如，对于 Adam 优化器、32 位权重以及一阶和二阶矩估计）在进程之间进行分区，以便每个进程仅更新其分区中的那一部分。

2. **ZeRO Stage 2**：用于更新模型权重的减少的 32 位梯度也被分区，这样每个进程只保留与其优化器状态部分对应的梯度。

3. **ZeRO Stage 3**：16 位模型参数跨进程分区。 ZeRO-3 将在前向和后向传递过程中自动收集和划分它们。

此外，ZeRO-3 包含infinity offload engine以形成 ZeRO-Infinity（[论文](https://arxiv.org/abs/2104.07857)），它可以将所有模型状态卸载到 CPU 和 NVMe 内存，以获得巨大的内存节约。如需深入了解算法，可以参阅有关论文。

### ZeRO config

更改ZeRO优化器各种设置的一个重要文件便是其config文件。完整config选项可见源码deepspeed.runtime.zero.config.DeepSpeedZeroConfig文件，这里只列举比较重要的几项：

In [ ]:
class ZeroStageEnum(int, Enum):
    """ Enum class for possible zero stages """
    disabled = 0
    optimizer_states = 1
    gradients = 2
    weights = 3
    max_stage = 3

class DeepSpeedZeroConfig(DeepSpeedConfigModel):
    """
    Sets parameters for ZeRO optimizations.
    """

    stage: ZeroStageEnum = 0
    """
    0：关闭  1：划分优化器状态  2：划分优化器状态和梯度状态  3：划分优化器状态、梯度状态和参数状态
    """

    contiguous_gradients: bool = True
    """
    将梯度复制到连续缓冲区中，以便在反向传播过程中避免内存碎片
    """

    reduce_scatter: bool = True
    """
    在梯度平均时，使用reduce或reduce scatter而不是allreduce
    """

    offload_param: Optional[DeepSpeedZeroOffloadParamConfig] = None
    """
    启用将模型参数卸载到 CPU 或 NVMe。这可以为更大的模型或批量大小释放 GPU 内存。仅对Stage 3 有效。
    """

    offload_optimizer: Optional[DeepSpeedZeroOffloadOptimizerConfig] = None
    """
    启用将优化器状态卸载到 CPU 或 NVMe。这可以为更大的模型或批量大小释放 GPU 内存。对Stage 1、2、3 有效。
    """


    gather_16bit_weights_on_model_save: bool = Field(False, alias="stage3_gather_16bit_weights_on_model_save")
    """
    在通过 save_16bit_model() 保存模型之前合并权重。由于权重是跨 GPU 分区的，它们不是 state_dict 的一部分，因此此函数会在启用此选项时自动收集权重，然后保存 fp16 模型权重。
    """

    # Validators
    @validator("overlap_comm")
    def overlap_comm_valid(cls, field_value, values):
        if field_value is None:
            assert ("stage" in values), "DeepSpeedZeroConfig: 'stage' must be defined before 'overlap_comm'"
            field_value = values["stage"] == ZeroStageEnum.weights
        return field_value

关于ZeRO config，还有关于Offload的两个类：

`classdeepspeed.runtime.zero.config.DeepSpeedZeroOffloadParamConfig`

`classdeepspeed.runtime.zero.config.DeepSpeedZeroOffloadOptimizerConfig`

分别是关于参数和优化器状态的offload，这里先忽略之

在config.json文件里面所撰写的ZeRO优化器设置大致包括：

```json
  "zero_optimization": {
    "stage": [0|1|2|3],
    "allgather_partitions": [true|false],
    "allgather_bucket_size": 5e8,
    "overlap_comm": false,
    "reduce_scatter": [true|false],
    "reduce_bucket_size": 5e8,
    "contiguous_gradients" : [true|false],
    "offload_param": {
      ...
    },
    "offload_optimizer": {
      ...
    },
    "stage3_max_live_parameters" : 1e9,
    "stage3_max_reuse_distance" : 1e9,
    "stage3_prefetch_bucket_size" : 5e8,
    "stage3_param_persistence_threshold" : 1e6,
    "sub_group_size" : 1e12,
    "elastic_checkpoint" : [true|false],
    "stage3_gather_16bit_weights_on_model_save": [true|false],
    "ignore_unused_parameters": [true|false]
    "round_robin_gradients": [true|false]
    }
```

### ZeRO Debugging

如果想要在debug的过程中访问模型参数、梯度和优化器状态，DeepSpeed 提供了以下API来访问他们，其中各参数都是以未被分区的形式来呈现的。

**重要提示**：请注意，参加training所有进程都必须调用这些实用程序，即使我们只想在主进程中对结果执行某些操作。如果没有所有进程都参与，这些API将被挂起。

此外，必须在特定的阶段访问正确的参数。例如，梯度在`backward`之后、`step`之前有效。优化器状态在`step`之后更新。 fp32 主权重也是如此。

In [ ]:
# in deepspeed/utils/tensor_fragment.py

# 收集并访问各分区fp16参数，返回fp32参数
def safe_get_full_fp32_param(param):
    """Assemble and return the fp32 parameter of a low-precision (e.g., fp16) parameter.

        Args:
            param (``torch.nn.Parameter``): A model parameter
    """
    # ZeRO stage 3 param
    if hasattr(param, 'ds_id'):
        return param._z3_optimizer.get_full_hp_param(param)

    # ZeRO stage 1, 2, and bf16_optimizer params
    if hasattr(param, '_hp_mapping'):
        return param.get_full_hp_param()
    return None


# 收集并访问各分区fp16优化器状态量，返回FP32优化器状态量
def safe_get_full_optimizer_state(param, optim_state_key):
    """Assemble and return the fp32 optimizer state of a low-precision (e.g., fp16) parameter.

        Args:
            param (``torch.nn.Parameter``): A model parameter
    """
    # ZeRO stage 3 param
    if hasattr(param, 'ds_id'):
        return param._z3_optimizer.get_full_hp_param(param, optim_state_key)

    # ZeRO stage 1, 2, and bf16_optimizer params
    if hasattr(param, '_hp_mapping'):
        return param.get_full_hp_param(optim_state_key)
    return None



# 收集并访问各分区fp16梯度，返回fp32梯度
# TODO: Figure out the correct return dtype
def safe_get_full_grad(param):
    """Assemble and return the fp32 gradient of a low-precision (e.g., fp16) parameter.

        Args:
            param (``torch.nn.Parameter``): A model parameter
    """
    if param.grad is not None:
        return param.grad

    # ZeRO stage 3 param
    if hasattr(param, 'ds_id'):
        return param._z3_optimizer.get_fp32_grad_for_param(param)

    # ZeRO stage 1, 2, and bf16_optimizer params
    if hasattr(param, '_hp_mapping'):
        return param.get_full_hp_grad()

    return None

将上述debug API应用在训练循环中的示例代码：

```py
backward(loss)
[...]
from deepspeed.utils import safe_get_full_fp32_param, safe_get_full_grad, safe_get_full_optimizer_state
for n, lp in model.named_parameters():
    # 1. gradient lookup
    # For zero1 and zero2, gradient lookup must be called after `backward` and before `step`
    # For zero3, gradient lookup must be called after `backward`
    hp_grad = safe_get_full_grad(lp)

    # 2. fp32 and optim states can probably be called anywhere in the training loop, but will be updated after `step`
    hp = safe_get_full_fp32_param(lp)
    exp_avg = safe_get_full_optimizer_state(lp, "exp_avg")
    exp_avg_sq = safe_get_full_optimizer_state(lp, "exp_avg_sq")

[...]
optimizer.step()
```

接下来，我们针对deepspeed/deepspeed/runtime/zero/stage_1_and_2.py这个文件里面最主要的

`class DeepSpeedZeroOptimizer(ZeROOptimizer):`

这个类（一共2000多行）的部分函数，分析一下具体的ZeRO优化器的计算是如何进行的

`init`部分先略过，首先看`def initialize_optimizer_states(self):`这个函数：

In [ ]:
def initialize_optimizer_states(self):

    # 先初始化优化器内参数（single_partition_of_fp32_groups）的梯度
    for i, group in enumerate(self.bit16_groups):
        single_grad_partition = torch.zeros(int(self.partition_size[i]),
                                            dtype=self.single_partition_of_fp32_groups[i].dtype,
                                            device=self.device)
        self.single_partition_of_fp32_groups[i].grad = get_accelerator().pin_memory(
            single_grad_partition) if self.cpu_offload else single_grad_partition

    # 核心是初始化优化器状态量时，更新一下self.optimizer
    self.optimizer.step()

    # 如果不需要进行cpu卸载，那么就将梯度置为None，即执行梯度清零
    if not self.cpu_offload:
        for group in self.single_partition_of_fp32_groups:
            group.grad = None  #class init

    return

接下来是ZeRO-Stage1划分梯度的做法：

In [ ]:
# Line 622
#########################################################################
#################### ZeRO Stage 1 - reduce gradients ####################
#########################################################################
def reduce_gradients(self, pipeline_parallel=False):
    world_size = dist.get_world_size(self.dp_process_group)
    my_rank = dist.get_rank(self.dp_process_group)

    # 使用Pipeline parellel时必须创建ipg_buffer，因为反向处理是在zero之外处理的
    if pipeline_parallel and self.contiguous_gradients:
        self.ipg_buffer = []
        buf_0 = torch.empty(int(self.reduce_bucket_size),
                            dtype=self.dtype,
                            device=get_accelerator().current_device_name())
        self.ipg_buffer.append(buf_0)
        self.ipg_index = 0

    if not self.overlap_comm:
        for i, group in enumerate(self.bit16_groups):
            for param in group:
                if param.grad is not None:
                    # 遍历每个分区的梯度，将其累加到对应的分区中
                    self.reduce_ready_partitions_and_remove_grads(param, i)
    # 在hook或是non-hook状态下都需要减少所有的pending待定状态下的梯度
    self.overlapping_partition_gradients_reduce_epilogue()
    
# ---------------------上面过程所用到的对应函数（套娃开始）---------------------
# Line 1250
def reduce_ready_partitions_and_remove_grads(self, param, i):
    if self.partition_gradients or self.is_gradient_accumulation_boundary:
        self.reduce_independent_p_g_buckets_and_remove_grads(param, i)
        
# Line 826
############### Independent Partition Gradient ########################
def reduce_independent_p_g_buckets_and_remove_grads(self, param, i):
    # 如果将要加入到 IPG 缓冲区中的梯度的大小和已经在缓冲区中的梯度的大小之和超过了预设的阈值 reduce_bucket_size，则需要对缓冲区中的梯度进行聚合，以释放一部分内存空间。
    if self.elements_in_ipg_bucket + param.numel() > self.reduce_bucket_size:
        self.report_ipg_memory_usage("In ipg_remove_grads before reduce_ipg_grads", param.numel())
        self.reduce_ipg_grads()
        if self.contiguous_gradients and self.overlap_comm:
            # Swap ipg_index between 0 and 1
            self.ipg_index = 1 - self.ipg_index
        self.report_ipg_memory_usage("In ipg_remove_grads after reduce_ipg_grads", param.numel())

    param_id = self.get_param_id(param)
    assert self.params_already_reduced[param_id] == False, \
        f"The parameter {param_id} has already been reduced. \
        Gradient computed twice for this partition. \
        Multiple gradient reduction is currently not supported"
        
    # 如果当前参数的大小超过了 reduce_bucket_size，则认为它是一个特别大的参数，不能加入到 IPG 缓冲区中，而是直接聚合。将它记录在 extra_large_param_to_reduce 变量中，在后续的 backward() 方法中会直接处理。
    if param.numel() > self.reduce_bucket_size:
        self.extra_large_param_to_reduce = param
    # 如果当前参数的大小小于等于 reduce_bucket_size，并且使用连续的梯度缓冲区，则将参数的梯度添加到 IPG 缓冲区的当前索引处，并更新缓冲区中已经存储的梯度的大小。这里使用了 narrow() 方法来从 IPG 缓冲区的当前索引处开始，连续地取出与参数梯度大小相等的一段空间，并将参数梯度的数据复制到这个空间中。这样可以避免内存碎片和梯度展开的问题，同时也能保证梯度在内存中的连续性，以便后续的通信和聚合操作。
    elif self.contiguous_gradients:
        # keeping the gradients contiguous to prevent memory fragmentation, and avoid flattening
        new_grad_tensor = self.ipg_buffer[self.ipg_index].narrow(0, self.elements_in_ipg_bucket, param.numel())
        new_grad_tensor.copy_(param.grad.view(-1))
        param.grad.data = new_grad_tensor.data.view_as(param.grad)
    # ------上面都是一些检查和准备的过程------
    
    # ------下面是将梯度加入到 IPG 缓冲区中------
    # 记录当前缓冲区中已经存储的梯度的大小
    self.elements_in_ipg_bucket += param.numel()
    
    # 如果参数的梯度是 None，则抛出异常，因为无法将 None 梯度加入到 IPG 缓冲区中。
    assert param.grad is not None, f"rank {dist.get_rank()} - Invalid to reduce Param {param_id} with None gradient"

    # 接下来，将参数的梯度添加到 grads_in_ipg_bucket 列表中，并将参数本身和其 ID 添加到 params_in_ipg_bucket 列表中。这两个列表用于在后续的 reduce_ipg_grads() 方法中对梯度进行聚合。
    self.grads_in_ipg_bucket.append(param.grad)
    self.params_in_ipg_bucket.append((i, param, param_id))

    # 给MOE模型的特殊参数做标记
    if is_moe_param(param):
        self.ipg_bucket_has_moe_params = True

    # 最后，调用 report_ipg_memory_usage() 方法记录 IPG 缓冲区的内存使用情况
    self.report_ipg_memory_usage("End ipg_remove_grads", 0)
    
# Line 815
def report_ipg_memory_usage(self, tag, param_elems):
    elem_count = self.elements_in_ipg_bucket + param_elems
    percent_of_bucket_size = (100.0 * elem_count) // self.reduce_bucket_size
    see_memory_usage(
        f"{tag}: elems in_bucket {self.elements_in_ipg_bucket} param {param_elems} max_percent {percent_of_bucket_size}"
    )